# 我们需要的数据


模型是怎么会说人话的呢？
借助海量数据做预训练而学
这一步，模型学知识。理解语义。

未经预训练的模型会输出什么呢？
"奥萨蒂ask的怀旧啊士大夫和"
这种乱打出来的字.
预训练是大批有语法有逻辑的语句，
从中模型可以学到有逻辑的话语



## 预训练数据
数据来源要尽可能广、多，才能保证这一阶段模型是在学习“通用知识”
预训练数据来自图书、代码、社交平台
我们先看看预训练数据应该长什么样，再思考如何处理

In [2]:
import json
import itertools
import jsonlines
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('../model/')

In [2]:
with jsonlines.open('../data/pretrain_hq.jsonl') as reader:
    data = list(itertools.islice(reader,5))
data

[{'text': '<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'},
 {'text': '<|im_start|>根据输入的内容，编写一个类别标签。\n这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编

可以看到
预训练所用的数据：
{"text": "这里是第一段文本内容...", "other_key": "其他信息..."}
text中包含的文本可以说，非常多样
我们要构造自己的数据集，就是把text提取出来，很简单


下面我们要把数据转化为模型看得懂的
这次我们先全部加载

In [3]:
samples = []
with open('../data/pretrain_hq.jsonl','r',encoding='utf-8') as f:
    for line in f:
        data = json.loads(line.strip())
        samples.append(data)
    


In [4]:
# 打印看看
samples

[{'text': '<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'},
 {'text': '<|im_start|>根据输入的内容，编写一个类别标签。\n这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编

那么现在已经成功加载，
由于过于庞大，我们抽取一部分进行后续操作。
回到正题，什么样的数据大模型是看的懂的？
给初高中学生讲的话，就是数字嘛！
但是这个数字也是有讲究的。
究竟把这些文本变成怎样的数字，详细看tokenizer这部分的讲解
我们要知道的是，tokenizer就是用来把刚才我们能看懂但机器看不懂的文本，变成机器能看懂我们看不懂的数字。
同时，要明白tokenizer处理文本后，所产出数据的结构：
- input_ids
- token_type_ids 
- attention_mask 
这些都是数字，如input_ids=[101, 7592, 102, 0, 0]
token_type_ids 标识 Token 属于哪个句子（用于区分句子 A 和句子 B，如 BERT 的双句子输入）
attention_mask告诉模型哪些位置是真实Token，哪些是填充符（Padding）

In [5]:
test_data = samples[:100]

简单理解，tokenizer把文字转化为数字，转化为的就是token
我们可以查看model文件夹里tokenizer.json文件，
里面有词典，即文字与数字的映射
比如特殊字符<|im_end|>  就被映射成 ‘2’

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('../model/')

In [17]:
text = "Hello<|im_end|>World"
input_ids = tokenizer.encode(text) 
input_ids

[42, 392, 338, 2, 57, 279, 512]

In [32]:
encoding  = tokenizer( "Hello<|im_end|>World. I’m man")
encoding

{'input_ids': [42, 392, 338, 2, 57, 279, 512, 16, 384, 1860, 79, 769], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

当然，tokenizer能填很多参数
其中'padding'  使得无论如何，都有预定的长度。就那么多字，你让我那么长？那就补0补齐。
现在我们使用预训练数据举例

In [26]:
sample = test_data[1]
sample

{'text': '<|im_start|>根据输入的内容，编写一个类别标签。\n这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要免费软件还是愿意付费？<|im_end|> <|im_start|>为什么我的程序不输出正确结果？可能是代码逻辑有误，或者输入数据有误，需要仔细调试代码逻辑和输入数据。<|im_end|> <|im_start|>谢谢你的回答。现在我想知道这场比赛的具体时间和地点。这场比赛的时间是北京时间10月4日，地点是上海。<|im_end|>'}

In [33]:
encoding_2 = tokenizer(
                     sample['text'],
                     max_length=512,
                     padding='max_length',
                     truncation=True,
                     return_tensors='pt')


In [35]:
encoding_2
# 有没有注意，往上翻一下encoding，看和这个有什么区别，都是哪些参数造成的？

{'input_ids': tensor([[   1, 1227, 2346, 5914,  270, 6095,  801, 1415, 1232, 5354,  286,  201,
          434, 1157, 2042, 2504, 1422, 3458,  854,  888, 2537, 5371,  801, 1415,
         1232, 5354,   28,  223, 2037,  595,   17,  854,  888, 2537, 3458, 5923,
            2,  223,    1, 1086,  397, 5885, 2336, 3694,  269, 3858, 1994,  286,
         6243,  270, 5213, 1086,  803, 5885,  682, 2711, 3858, 1994,  286, 4600,
          803,  590, 1698, 3445, 2404,  371, 1971,  269, 3858, 1994, 3274,  814,
            2,  223,    1, 1086,  397, 2910,  541, 5804,  953,  854, 5221,  242,
         1531,  286, 1188,  270,  397, 1086,  803, 2910,  541, 2345, 1439, 1330,
          237, 5221,  242, 1531,  286, 4587, 1439, 1330,  237, 3993,  568, 1687,
          814, 1775, 4874, 1170,  345, 1476,  953,  446, 2207,    2,  223,    1,
         2525, 4584, 2706,  333, 1581, 2345,  611, 1781,  269, 3044, 1486, 3411,
          286, 1944,  103, 3344, 3139,  876,  118, 1749, 1729,  954,  270, 1781,
          954,


loss_mask：

生成一个布尔掩码，指定哪些位置的预测需要计算损失，哪些位置忽略。
这是可以自定义的，attentionmask是定死的padding位置忽略，
attention_mask 控制模型“看哪里”，但无法阻止损失函数计算某些位置的梯度。
例如，若 pad_token_id=0，则 input_ids=[101, 7592, 102, 0, 0] → loss_mask=[True, True, True, False, False]。

In [39]:

loss_mask = (input_ids != tokenizer.pad_token_id)

tensor([   1, 1227, 2346, 5914,  270, 6095,  801, 1415, 1232, 5354,  286,  201,
         434, 1157, 2042, 2504, 1422, 3458,  854,  888, 2537, 5371,  801, 1415,
        1232, 5354,   28,  223, 2037,  595,   17,  854,  888, 2537, 3458, 5923,
           2,  223,    1, 1086,  397, 5885, 2336, 3694,  269, 3858, 1994,  286,
        6243,  270, 5213, 1086,  803, 5885,  682, 2711, 3858, 1994,  286, 4600,
         803,  590, 1698, 3445, 2404,  371, 1971,  269, 3858, 1994, 3274,  814,
           2,  223,    1, 1086,  397, 2910,  541, 5804,  953,  854, 5221,  242,
        1531,  286, 1188,  270,  397, 1086,  803, 2910,  541, 2345, 1439, 1330,
         237, 5221,  242, 1531,  286, 4587, 1439, 1330,  237, 3993,  568, 1687,
         814, 1775, 4874, 1170,  345, 1476,  953,  446, 2207,    2,  223,    1,
        2525, 4584, 2706,  333, 1581, 2345,  611, 1781,  269, 3044, 1486, 3411,
         286, 1944,  103, 3344, 3139,  876,  118, 1749, 1729,  954,  270, 1781,
         954,  845, 1468, 1319,  954,  5

大模型的核心逻辑是预测！我们要让模型学会预测下一个词
input_ids = [A, B, C, D]  # 原始序列

*构造训练对*：
X = [A, B, C]   # 输入序列 (位置 0~2)
Y = [B, C, D]   # 目标序列 (位置 1~3)

*模型训练任务*：
- 输入 A → 预测 B ✅
- 输入 A,B → 预测 C ✅
- 输入 A,B,C → 预测 D ✅

In [41]:
X = torch.tensor(input_ids[:-1], dtype=torch.long)
Y = torch.tensor(input_ids[1:], dtype=torch.long)
loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)

/tmp/ipykernel_85545/2823934175.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)


In [42]:
X

tensor([ 42, 392, 338,   2,  57, 279])

In [43]:
Y

tensor([392, 338,   2,  57, 279, 512])

In [44]:
loss_mask

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

.squeeze()方法return_tensor这个参数有关
把上述方法封装成函数如下

In [ ]:
class PretrainDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(data_path)

    def load_data(self,data_path):
        samples = []
        with open(data_path,'r',encoding='utf-8') as f:
            for line in f:
                data = line.strip()
                samples.append(data)
        return samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index) :
        sample = self.samples[index]

        encoding = self.tokenizer(
            sample[index],
            padding = self.max_length,
            max_length = self.max_length,
            truncation = True,
            return_tensor = 'pt'
        )

        input_ids = encoding.input_ids.squeeze()
        loss_mask = (input_ids != self.tokenizer.pad_token_id)

        X = torch.tensor(input_ids[:-1],dtype=torch.long)
        Y = torch.tensor(input_ids[1:],dtype=torch.long)

        return loss_mask,X,Y

        


# SFT流程
SFT 的全称是 Supervised Fine-Tuning，即 监督微调
在预训练阶段，模型在海量的、无标签的文本数据上进行训练，学习的目标通常是“预测下一个词”。这使得模型掌握了语言的语法、事实知识和基本的推理能力。 但此时的模型更像一个知识渊博但不懂得如何交流的“书呆子”，它只会根据上文补全句子，而不会特意地“回答问题”或“遵循指令”。
SFT阶段就是要把这个“书呆子”训练成一个乐于助人、善于沟通的AI助手。 这个过程就像是给模型一本高质量的“问答练习册”（即SFT数据集），其中包含了大量的“指令-回答”范例。

看看数据长什么样

In [4]:
with jsonlines.open('../data/sft_1024.jsonl') as reader:
    data = list(itertools.islice(reader,1))
data

[{'conversations': [{'role': 'user',
    'content': 'What is the most recent version of the Android operating system?'},
   {'role': 'assistant',
    'content': 'As of my last update in October 2023, the most recent version of the Android operating system is Android 14. It was officially released in August 2023. Android 14 brings several new features and improvements, including enhanced privacy controls, performance optimizations, and user interface refinements. For the most current information, you can check the official Android website or the latest announcements from Google.'}]}]

但这个数据还不是直接输入到模型的！因为SFT格式很多的，
- instruction、input、output
- q、a
等等
那么这个数据要转化成什么样，模型才能很好的学会应答呢？
这跟学的tokenizer有关了
我们找找tokenizer_config文件，发现有如下一段配置

```
  "additional_special_tokens": [],
  "bos_token": "<|im_start|>",
  "clean_up_tokenization_spaces": false,
  "eos_token": "<|im_end|>",
  "legacy": true,
  "model_max_length": 32768,
  "pad_token": "<|endoftext|>",
  "sp_model_kwargs": {},
  "spaces_between_special_tokens": false,
  "tokenizer_class": "PreTrainedTokenizerFast",
  "unk_token": "<|endoftext|>",
  "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{{ '<|im_start|>system\\n' + system_message + '<|im_end|>\\n' }}{% else %}{{ '<|im_start|>system\\nYou are a helpful assistant<|im_end|>\\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"

```
下面介绍一下
- bos_token (Begin of Sequence): 定义了哪个标记代表“序列开始”。在这里是 <|im_start|>。
- eos_token (End of Sequence): 定义了哪个标记代表“序列结束”。在这里是 <|im_end|>。这是模型知道何时该停止生成文本的关键信号。
- pad_token (Padding Token): 在批处理（batch processing）中，为了让所有序列长度一致，需要用一个特殊标记来填充较短的序列。这里用 <|endoftext|> 来填充。
- unk_token (Unknown Token): 当遇到一个词汇表里没有的词时，用这个标记来代替。
- model_max_length: 模型能处理的最大序列长度（上下文窗口），这里是32768个Token。
- chat_template: 这是最关键的部分。它是一个用 Jinja2 模板语言编写的“小程序”，定义了一个精确的“食谱”，告诉分词器如何将一个对话列表（比如 [{'role': 'user', ...}, ...]）转换成一个最终的、可以被模型处理的字符串。

## 介绍chat_template

检查对话列表的第一个消息是不是system角色。

如果是，就把system的内容拿出来，包装成 <|im_start|>system\n...内容...<|im_end|>\n 的格式。

如果不是（大部分SFT数据都没有system角色），它会自动添加一个默认的系统提示：<|im_start|>system\nYou are a helpful assistant<|im_end|>\n。

这就是为什么很多模型默认表现得像一个“乐于助人的助手”的原因

当遇到 user 消息时: 它会生成：

起始标记和角色：<|im_start|>user\n

用户说的内容：content

结束标记：<|im_end|>\n

最关键的一步: 它会立刻紧跟着输出一个“提示”，告诉模型该轮到它发言了：<|im_start|>assistant\n

当遇到 assistant 消息时: 它会生成：
1. 助手说的内容：content
2. 结束标记：<|im_end|>\n
chat_template` 是一个自动化、标准化的“对话格式化引擎”。它确保了无论原始数据长什么样，只要是`[{'role': ..., 'content': ...}]`的格式，都能被准确无误地转换成模型在训练时所学习和理解的、带有特殊标记的唯一格式

简而言之，收集的SFT数据集类似这样
```messages = [
    {"role": "user", "content": "你好"},
    {"role": "assistant", "content": "你好！有什么可以帮助你的吗？"}
]
```
我们转化为如下格式
```
<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n你好<|im_end|>\n<|im_start|>assistant\n你好！有什么可以帮助你的吗？<|im_end|>\n```


In [3]:
with jsonlines.open('../data/sft_1024.jsonl') as reader:
    data2 = list(itertools.islice(reader,4))
data2

[{'conversations': [{'role': 'user',
    'content': 'What is the most recent version of the Android operating system?'},
   {'role': 'assistant',
    'content': 'As of my last update in October 2023, the most recent version of the Android operating system is Android 14. It was officially released in August 2023. Android 14 brings several new features and improvements, including enhanced privacy controls, performance optimizations, and user interface refinements. For the most current information, you can check the official Android website or the latest announcements from Google.'}]},
 {'conversations': [{'role': 'user',
    'content': 'How can I generate a random number in Python between 1 and 100, inclusive?'},
   {'role': 'assistant',
    'content': 'To generate a random number between 1 and 100 (inclusive) in Python, you can use the `random` module, which provides various functions to generate random numbers. Here’s how you can do it:\n\n```python\nimport random\n\n# Generate a rando

```
{'conversations':
   [ 
    {'role': 'user',
    'content': 'What is the most recent version of the Android operating system?'},
   {'role': 'assistant',
    'content': 'As of my last updaxxxx'}  
    ] 
}
```
我们要的是
```
   {'role': 'user',
    'content': 'What is the most recent version of the Android operating system?'},
   {'role': 'assistant',
    'content': 'As of my last updaxxxx'} 
```

In [4]:
data2[1]['conversations']

[{'role': 'user',
  'content': 'How can I generate a random number in Python between 1 and 100, inclusive?'},
 {'role': 'assistant',
  'content': 'To generate a random number between 1 and 100 (inclusive) in Python, you can use the `random` module, which provides various functions to generate random numbers. Here’s how you can do it:\n\n```python\nimport random\n\n# Generate a random integer between 1 and 100, inclusive\nrandom_number = random.randint(1, 100)\n\nprint(random_number)\n```\n\n### Explanation:\n- `random.randint(a, b)` returns a random integer `N` such that `a <= N <= b`. This means it includes both the endpoints `a` and `b`.\n\nThis code will print a random integer between 1 and 100 each time you run it.'}]

In [5]:
messages = []
for i, turn in enumerate(data2[1]['conversations']):
    role = 'user' if i % 2 == 0 else 'assistant'
    messages.append({"role": role, "content": turn['content']})

这里不是多此一举，而是展示如何构造数据集的
如果源数据没有user、assistant这些提示词呢？

In [6]:
messages

[{'role': 'user',
  'content': 'How can I generate a random number in Python between 1 and 100, inclusive?'},
 {'role': 'assistant',
  'content': 'To generate a random number between 1 and 100 (inclusive) in Python, you can use the `random` module, which provides various functions to generate random numbers. Here’s how you can do it:\n\n```python\nimport random\n\n# Generate a random integer between 1 and 100, inclusive\nrandom_number = random.randint(1, 100)\n\nprint(random_number)\n```\n\n### Explanation:\n- `random.randint(a, b)` returns a random integer `N` such that `a <= N <= b`. This means it includes both the endpoints `a` and `b`.\n\nThis code will print a random integer between 1 and 100 each time you run it.'}]

In [7]:
prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

In [8]:
prompt

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\nHow can I generate a random number in Python between 1 and 100, inclusive?<|im_end|>\n<|im_start|>assistant\nTo generate a random number between 1 and 100 (inclusive) in Python, you can use the `random` module, which provides various functions to generate random numbers. Here’s how you can do it:\n\n```python\nimport random\n\n# Generate a random integer between 1 and 100, inclusive\nrandom_number = random.randint(1, 100)\n\nprint(random_number)\n```\n\n### Explanation:\n- `random.randint(a, b)` returns a random integer `N` such that `a <= N <= b`. This means it includes both the endpoints `a` and `b`.\n\nThis code will print a random integer between 1 and 100 each time you run it.<|im_end|>\n'

In [13]:
input_ids = tokenizer(prompt,
                     max_length=512,
                     padding='max_length',
                     truncation=True,
                     return_tensors='pt').input_ids.squeeze()

In [14]:
input_ids

tensor([   1,   85,  736,  201,   59,  292,  389,  260, 3836, 1861,  501,    2,
         201,    1,  320,  275,  201, 1936,  343,  384, 4451,  260, 1482,  273,
         327, 3717,  295,  614, 2143, 1833,  730,  281, 5171,   14, 6194,   33,
           2,  201,    1, 1078,  538,  501,  201, 5042, 4451,  260, 1482,  273,
         327, 3717, 1833,  730,  281, 5171,  958,  261, 4109, 4941,   11,  295,
         614, 2143,   14,  363,  343,  813,  276, 3745, 3830,  327,   66,  461,
        6139,   14,  896, 3198, 2514, 2356,  325,  524,  290, 4451, 1482,  273,
         327, 1921, 2781,   16, 1193, 1860,   85,  822,  363,  343, 1004,  470,
          28,  201,  201, 1638, 4751,  201,  334, 1519, 1482,  273,  327,  201,
         201,    5,  886, 3877,  429,  260, 1482,  273,  327,  696, 1841,  275,
        1833,  730,  281, 5171,   14, 6194,  201, 3830,  327,   65, 3267, 1485,
         901, 1482,  273,  327,   16, 3830, 1131,   10,   19,   14, 5171,   11,
         201,  201, 2809,   10, 3830,  3

In [21]:
loss_mask = len(input_ids)*[0]
bos_id = tokenizer('<|im_start|>assistant', add_special_tokens=False).input_ids
eos_id = tokenizer('<|im_end|>', add_special_tokens=False).input_ids

while i < len(input_ids):
    if input_ids[i:i+len(bos_id)] == bos_id:
        start = i + len(bos_id) 
         
        try:
            # 使用 list.index() 来高效查找 eos_id 的起始位置
            # 我们只在 bos_id 之后的部分查找
            end_token_index = input_ids.index(eos_id[0], start)
            
            # 确认这确实是一个完整的 eos_id 序列
            if input_ids[end_token_index:end_token_index+len(eos_id)] == eos_id:
                # 标记从回复开始到结束标记之前的所有 token
                # 通常我们希望模型学会预测 eos_id，所以也包含它
                end_of_response = end_token_index + len(eos_id)
                for j in range(start, end_of_response):
                    loss_mask[j] = 1
                
                # 将主索引 i 直接跳转到处理完的这段序列末尾
                i = end_of_response
            else:
                # 如果 index 找到的不是完整的 eos_id，则正常递增 i
                i += 1
        except ValueError:
            # 如果在 bos_id 之后找不到 eos_id (例如，序列被截断)
            # 那么将从回复开始到序列末尾的所有 token 都进行标记
            for j in range(start, len(input_ids)):
                loss_mask[j] = 1
            # 已经处理到末尾，可以结束循环
            break
    else:
            # 如果当前位置不是起始标记，则索引向后移动一位
        i += 1



[2]

In [22]:
loss_mask

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


对于lossmask函数，就是借助bosid和eosid 找到assisatant输出部分 让他们的lossmask部分等于1.  后面序列化
我们已经稍微熟悉tokenizer了
```input_ids = self.tokenizer(prompt).input_ids[:self.max_length]
input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids))
```
和
```
input_ids = tokenizer(prompt,
                     max_length=512,
                     padding='max_length',
                     truncation=True,
                     return_tensors='pt')
```
效果是一样的哦

In [ ]:
class SFTDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_length=1024):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(jsonl_path)
        self.bos_id = tokenizer('<|im_start|>assistant', add_special_tokens=False).input_ids
        self.eos_id = tokenizer('<|im_end|>', add_special_tokens=False).input_ids

    def __len__(self):
        return len(self.samples)

    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples

    def _create_chat_prompt(self, conversations):
        """构建符合ChatML格式的对话"""
        messages = []
        for i, turn in enumerate(conversations):
            role = 'user' if i % 2 == 0 else 'assistant'
            messages.append({"role": role, "content": turn['content']})
        return self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

    def _generate_loss_mask(self, input_ids):
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id)
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)):
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask

    def __getitem__(self, index):
        sample = self.samples[index]
        # 构建对话提示
        prompt = self._create_chat_prompt(sample['conversations'])
        input_ids = self.tokenizer(prompt).input_ids[:self.max_length]
        input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids))

        # 生成动态损失掩码
        loss_mask = self._generate_loss_mask(input_ids)

        # 构建训练数据
        X = torch.tensor(input_ids[:-1], dtype=torch.long)
        Y = torch.tensor(input_ids[1:], dtype=torch.long)
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)  # 对齐预测位置

        return X, Y, loss_mask

# RLHF
在经历SFT后，模型能回答人话了，但回答的质量嘛...

我们为了让他回答质量更好，需要再次进行训练

此时我们训练的方法是RLHF (Reinforcement Learning from Human Feedback - 基于人类反馈的强化学习)

1. 我们需要训练奖励模型，收集偏好数据，即人类觉得回答很棒的数据

2. 我们用这些数据集单独训练一个奖励模型，奖励模型用来给SFT训练好的模型输出打分，以分数作为奖励信号（想想预训练和SFT阶段我们的奖励信号是啥嘞），来更新SFT模型（这一步一般叫PPO算法）

-----

DPO的全称是：Direct Preference Optimization (直接偏好优化)。
我们不去搞奖励模型那么繁琐的ppo算法流程了，直接利用分割线第一步的偏好数据，直接在原SFT后模型中，直接进行优化
得到更好的DPO模型



看看数据长什么样

In [25]:
with jsonlines.open('../data/dpo.jsonl') as reader:
    data = list(itertools.islice(reader,1))
data

[{'chosen': [{'content': 'How many moles of HBr are required to react with 2 moles of C2H6 to form 2 moles of C2H5Br along with 2 moles of H2?',
    'role': 'user'},
   {'content': 'To determine the number of moles of HBr required to react with 2 moles of C2H6 (ethane) to form 2 moles of C2H5Br (ethyl bromide) and 2 moles of H2, we need to write the balanced chemical equation for the reaction.\n\nThe reaction is a halogenation reaction, specifically the reaction of ethane with hydrogen bromide (HBr) to form ethyl bromide and hydrogen gas. The balanced chemical equation for this reaction is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\rightarrow \\text{C}_2\\text{H}_5\\text{Br} + \\text{H}_2 \\]\n\nHowever, this equation is not balanced as written. To balance it, we need to ensure that the number of atoms of each element on the reactant side is equal to the number of atoms of that element on the product side. The balanced equation is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\righ

chosen意思是人类偏好的回答，是质量好的回答
rejected意思是人类拒绝的回答，质量差的回答

这里的预处理其实和SFT一样的，提取content和role，应用tokenizer.apply_chat_template
inputids是全部文本都要，lossmask是只将assistance的消息设置为1

# RLAIF
全称是 Reinforcement Learning from AI Feedback (基于AI反馈的强化学习)
使用的方法称为Distillation，蒸馏
蒸馏的本意是使用已经有的优秀模型，比如说开源的高参数模型，让它当作老师，对一些问题给出参考答案。我们正在训练的小模型，就可以针对参考答案去升级
在得到DPO模型后，可以把这个模型看作“优秀模型”，也就是“老师”
我们把老师生成的参考答案，再给模型学习一次，有监督训练。
也就是 PreTrain：学会通用知识，认识世界---SFT:学会回答人类问题---DPO：基于RLHF，在SFT模型基础上，训练一个优秀的教师模型，它回答人类问题的质量更高---Distillation：将DPO模型视作SFT模型的教师模型，进行蒸馏，所谓蒸馏仍是一种SFT，就是SFT上的再次SFT